In [1]:
import pandas as pd
coefficients = []

In [2]:
# %run data_processing.py
import numpy as np
import json
import string
import operator
from collections import *

class OrderedCounter(Counter, OrderedDict):
    pass

# array is 


"""
 Brainstormed features:
 - sentiment analysis
"""

def add_features(top_words, data):
    # Word count feature
    # 100 data points
    x_train = np.zeros((10000, 161))
    y_train = np.zeros(10000)
    i = 0
    for data_point in data:
        features = np.zeros(161)
        word_list = process_string(data_point['text'])
    
        for word in word_list:
            if word in top_words:
                index = top_words.index(word)
                features[index] += 1
        features[160] = 1
        x_train[i] = features
        y_train[i] = data_point["popularity_score"]
        i = i+1

    # TODO: at least two more features: these can be 
    # based on the text data, transformations of the other numeric features, or interaction terms. 
    return x_train, y_train

def calculate_closed_form(X, Y):
    coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(Y)
    return coeffs

def read_json_file():
    file = open('./data/proj1_data.json', 'r')
    data= file.read()
    file.close()
    json_data = json.loads(data)
    return json_data

def process_string(str):
    return str.lower().split(' ')

def count_top_words(data):
    print("Finding top words")
    word_freq = OrderedCounter()
    for line in data:
        word_list = process_string(line['text'])
        word_freq.update(word_list)
    most_common = dict(word_freq.most_common(160))
    return list(most_common.keys())

def split_data(data, first_split, second_split, third_split):
    train, validation, test = [], [], []

    for i in range(0, first_split):
        train.append(data[i])
    print(("Created training dataset. Sample from training data: {}").format(train[0]))

    for i in range(first_split, second_split):
        validation.append(data[i])
    print(("Created validation dataset. Sample from validation data: {}").format(validation[0]))

    for i in range(second_split, third_split):
        test.append(data[i]) 
    print(("Created test dataset. Sample from validation data: {}").format(test[0]))

    return train, validation, test

def main():
    data = read_json_file()

    """
    Use the first 10,000 points for training, the next 1,000 for validation, and the final 1,000 for testing.
    """
    train, validation, test = split_data(data, 10000, 11000, 12000)

    top_words = count_top_words(train)
    # print(top_words)
    
    data_with_features, y_train = add_features(top_words, train)
    coefficients = calculate_closed_form(data_with_features, y_train)
    return coefficients

if __name__ == '__main__':
    coefficients = main()

Created training dataset. Sample from training data: {u'is_root': False, u'text': u'ITS RAINING SIDEWAYS', u'popularity_score': 1.254698160267241, u'controversiality': 0, u'children': 0}
Created validation dataset. Sample from validation data: {u'is_root': True, u'text': u'Kidney infection and subsequent stones. ', u'popularity_score': 0.8433369681773519, u'controversiality': 0, u'children': 0}
Created test dataset. Sample from validation data: {u'is_root': True, u'text': u"Some asshole shot 9 innocent people at a church.  I live in such a beautiful city, and it sucks that this made national headlines.  Charleston is beautiful.  It's been proven over and over again.  You owe it to yourself to visit.", u'popularity_score': 0.7204745721237265, u'controversiality': 0, u'children': 1}
Finding top words


In [3]:
coefficients

array([-1.11812793e-02, -1.97377137e-02, -4.45503046e-03, -4.29237766e-02,
       -2.36255504e-02,  9.98394724e-02, -7.82411651e-02, -3.87072263e-01,
       -3.38520978e-02,  6.30789860e-02,  1.09819629e-01, -6.17252928e-02,
        4.17305065e-02, -9.08702703e-03,  2.59126027e-02, -1.80077060e-02,
        4.55601484e-02,  1.64503088e-01, -2.75914251e-02,  8.87745571e-02,
        1.58733039e-01, -1.33735780e-01,  7.37559241e-04, -8.14110232e-02,
       -2.17491648e-02,  3.00791566e-02,  4.15889554e-02,  1.32264336e-01,
        8.04462890e-02, -1.44723802e-02,  2.34401332e-02,  3.05711321e-02,
        2.82310895e-02,  6.57065798e-03, -2.05093882e-02, -4.43061696e-02,
       -1.54628443e-02, -8.29632438e-03,  8.96239145e-03,  2.23969588e-02,
        2.78214295e-02,  1.14427431e-01, -4.77994927e-02, -8.25836886e-04,
        1.79550540e-01, -1.03604442e-01,  3.11977638e-02,  2.03928484e-02,
       -4.95551502e-02,  3.03552320e-02,  4.59889340e-02, -6.44799031e-03,
       -8.21024874e-02,  